<a href="https://colab.research.google.com/github/CygnusST3RN/FHE.0/blob/ankita_ml/FHE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import importlib  
import bfv
bfv=importlib.reload(bfv)

Initialize BFV Parameters

In [2]:

params = bfv.Params(128, 32, 4293918721, 2**218)
fv12= bfv.FV12(params)
public_key,private_key=fv12.generate_keys()


Initialised
Eval Key:0/43 Done
Eval Key:1/43 Done
Eval Key:2/43 Done
Eval Key:3/43 Done
Eval Key:4/43 Done
Eval Key:5/43 Done
Eval Key:6/43 Done
Eval Key:7/43 Done
Eval Key:8/43 Done
Eval Key:9/43 Done
Eval Key:10/43 Done
Eval Key:11/43 Done
Eval Key:12/43 Done
Eval Key:13/43 Done
Eval Key:14/43 Done
Eval Key:15/43 Done
Eval Key:16/43 Done
Eval Key:17/43 Done
Eval Key:18/43 Done
Eval Key:19/43 Done
Eval Key:20/43 Done
Eval Key:21/43 Done
Eval Key:22/43 Done
Eval Key:23/43 Done
Eval Key:24/43 Done
Eval Key:25/43 Done
Eval Key:26/43 Done
Eval Key:27/43 Done
Eval Key:28/43 Done
Eval Key:29/43 Done
Eval Key:30/43 Done
Eval Key:31/43 Done
Eval Key:32/43 Done
Eval Key:33/43 Done
Eval Key:34/43 Done
Eval Key:35/43 Done
Eval Key:36/43 Done
Eval Key:37/43 Done
Eval Key:38/43 Done
Eval Key:39/43 Done
Eval Key:40/43 Done
Eval Key:41/43 Done
Eval Key:42/43 Done
Eval Key:43/43 Done


Scale Input Values Based on Plaintext modulus and read Dataset 

In [4]:
def read_dataset(url,col,scale):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = MinMaxScaler(feature_range=(0, scale))
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [5]:
X_train, X_test, y_train, y_test = read_dataset('./dataset/data.csv',1,1000)

(143, 30)


C:\Users\delwy\AppData\Local\Temp\ipykernel_22260\3440237989.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
C:\Users\delwy\AppData\Local\Temp\ipykernel_22260\3440237989.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


Train Model

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(2),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/30
27/27 [==============================] - 1s 2ms/step - loss: 116.0200 - accuracy: 0.3662
Epoch 2/30
27/27 [==============================] - 0s 2ms/step - loss: 91.5453 - accuracy: 0.4272
Epoch 3/30
27/27 [==============================] - 0s 2ms/step - loss: 71.1968 - accuracy: 0.4648
Epoch 4/30
27/27 [==============================] - 0s 2ms/step - loss: 54.2875 - accuracy: 0.5376
Epoch 5/30
27/27 [==============================] - 0s 2ms/step - loss: 43.3938 - accuracy: 0.6033
Epoch 6/30
27/27 [==============================] - 0s 2ms/step - loss: 34.7248 - accuracy: 0.6338
Epoch 7/30
27/27 [==============================] - 0s 2ms/step - loss: 27.4837 - accuracy: 0.6784
Epoch 8/30
27/27 [==============================] - 0s 2ms/step - loss: 22.5734 - accuracy: 0.6925
Epoch 9/30
27/27 [==============================] - 0s 2ms/step - loss: 18.1053 - accuracy: 0.7371
Epoch 10/30
27/27 [==============================] - 0s 2ms/step - loss: 16.1664 - accuracy: 0.7582
Epoch 11

Save Quantised Model Weights

In [14]:
def create_model_weights_file(model,scale):
  i=0
  for layer in model.layers:
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    w=np.round_(w*scale).astype(int)
    b=np.round_(b*scale).astype(int)
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [15]:
create_model_weights_file(model,100)

Custom class for inference on encrypted data


In [16]:
class Model:
    def __init__(self,n_features,n_layers,n_layer_neurons,layer_weight_files):
        self.n_features=n_features
        self.n_layers=n_layers
        self.n_layer_neurons=n_layer_neurons
        self.weights=[]
        self.biases=[]
        for wt_file in layer_weight_files:
            with open(wt_file, 'rb') as f:
              self.weights.append(np.load(f).tolist())
              self.biases.append(np.load(f).tolist())
    def __matmul(self,a,b):
        ## Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list 
        output=[]
        for i in range(len(a)):
            res=0
            for j in range(len(b)):
                res=a[i][j]*b[j]+res
            output.append(res)
        return output
    
    def __matadd(self,a,b):
        ## Input: a and b to be 1-D list
        # Output : a 1-D list 
        ## For memory saving b is modified in calculations so be careful about order of inputs
        
        for i in range(len(a)):
            b[i]=b[i]+a[i]
        return b  
    
    def infer(self,x,encrypted=True):
        for i in range(self.n_layers):
            
            new_x=self.__matmul(self.weights[i],x)
            new_x=self.__matadd(self.biases[i],new_x)
            del x
            x=new_x  
        return x[0]
            
        
        
        
        

In [17]:
model=Model(30,2,[2,1],['layer0.npy','layer1.npy'])

Test out accuracy on Encrypted and Unencrypted test data

In [18]:
normal_acc=0
encrypted_acc=0
for i in range(len(X_test)):
    x=X_test[i].tolist()
    x_encrypted=[public_key.encrypt(t) for t in x]
    if (model.infer(x,False)>0 and y_test[i]==1) or (model.infer(x,False)<=0 and y_test[i]==0):
        normal_acc+=1
    if (private_key.decrypt(model.infer(x_encrypted,False))>0 and y_test[i]==1) or (private_key.decrypt(model.infer(x_encrypted,False))<=0 and y_test[i]==0):
        encrypted_acc+=1
print('Accuracy on unenencrypted Test Data:'+str(normal_acc/len(X_test)))
print('Accuracy on encrypted test data:'+str(encrypted_acc/len(X_test)))
  

0.8741258741258742
0.8741258741258742
